import libraries

In [ ]:
import json
import gpxpy
from datetime import datetime

Options

helper functions

In [ ]:
def timeToDatetime(time):
    timeStampString = str(time)
    if timeStampString[-1] == 'Z':
        timeStampString = timeStampString[0:-1]
    timeExample = '0000-00-00T00:00:00.000'

    if len(timeStampString) < len(timeExample):
        timeStampString = timeStampString + timeExample[len(timeStampString):]
    elif len(timeStampString) > len(timeExample):
        timeStampString = timeStampString[0:len(timeExample)]
        
    timeStamp = datetime.strptime(timeStampString, '%Y-%m-%dT%H:%M:%S.%f')

    return timeStamp

def correctLongAndLat(value):
    return value / 10000000

In [ ]:
def loadOriginal(path, fileName):

    print('Loading original data...')
    with open(path + fileName, 'r') as f:
        data = json.load(f)

    print('Simplifying data...')
    simplifiedData = []
    for entry in data['locations']:
        simplifiedData.append({
            'latitude': entry['latitudeE7'] / 10000000,
            'longitude': entry['longitudeE7'] / 10000000,
            'time': entry['timestamp']
        })

    print('Saving simplified data...')
    with open(path + 'simplified-' + fileName, 'w') as f:
        json.dump(simplifiedData, f)

    print('----- Done -----')
    return simplifiedData


def loadSimplified(path, fileName):

    print('Loading simplified data...')
    with open(path + fileName, 'r') as f:
        data = json.load(f)

    print('----- Done -----')
    return data

In [ ]:
def splitData(data, splitInto):

    if splitInto == -1:
        return [{
            'id': 'all',
            'data': [data]
        }]

    print('Splitting data...')
    splittedData = []
    updateCounter = 0

    if splitInto == 0:  # split into years
        for entry in data:

            updateCounter += 1
            if updateCounter > 1000:
                print('...into ' + str(len(splittedData)) + ' years...', end='\r')
                updateCounter = 0

            entryTime = timeToDatetime(entry['time'])
            entryYear = entryTime.year

            if len(splittedData) == 0 or splittedData[-1]['id'] != entryYear:
                splittedData.append({
                    'id': entryYear,
                    'data': []
                })
            splittedData[-1]['data'].append(entry)

    elif splitInto == 1:  # split into months
        for entry in data:

            updateCounter += 1
            if updateCounter > 1000:
                print('...into ' + str(len(splittedData)) +
                      ' months...', end='\r')
                updateCounter = 0

            entryTime = timeToDatetime(entry['time'])
            entryYear = entryTime.year
            entryMonth = entryTime.month

            if len(splittedData) == 0 or splittedData[-1]['id'] != (entryYear, entryMonth):
                splittedData.append({
                    'id': (entryYear, entryMonth),
                    'data': []
                })
            splittedData[-1]['data'].append(entry)

    print('----- Done -----')
    return splittedData


In [ ]:
def createGfx(input, oneTrackPer):

    gpxArray = []
    print('Creating gpx...')

    for dataOrig in input:

        data = dataOrig['data']
        gpx = gpxpy.gpx.GPX()

        track = gpxpy.gpx.GPXTrack()
        gpx.tracks.append(track)

        segment = gpxpy.gpx.GPXTrackSegment()
        track.segments.append(segment)

        updateCounter = 0

        # split into tracks
        if oneTrackPer == -1:  # no split

            for entry in data:
                printOutput = '...{id} | track {current}/{total} with {points} points...'.format(
                    id=dataOrig['id'],
                    current=len(gpxArray) + 1,
                    total=len(input),
                    points=len(segment.points),
                )

                updateCounter += 1
                if updateCounter > 1000:
                    print(printOutput, end='\r')
                    updateCounter = 0

                segment.points.append(gpxpy.gpx.GPXTrackPoint(
                    correctLongAndLat(entry['latitude']),
                    correctLongAndLat(entry['longitude']),
                    time=timeToDatetime(entry['time'])
                ))

            print(printOutput)

        elif oneTrackPer == 0:  # day
            for entry in data:

                updateCounter += 1
                if updateCounter > 1000:
                    print('...{id} | {day} days...'.format(
                        id=dataOrig['id'],
                        day=len(track.segments),
                    ), end='\r')
                    updateCounter = 0

                if len(segment.points) == 0 or timeToDatetime(entry['time']).day != segment.points[-1].time.day:
                    segment = gpxpy.gpx.GPXTrackSegment()
                    track.segments.append(segment)
                segment.points.append(gpxpy.gpx.GPXTrackPoint(
                    correctLongAndLat(entry['latitude']),
                    correctLongAndLat(entry['longitude']),
                    time=timeToDatetime(entry['time'])
                ))
            print('...{id} | {day} days...'.format(
                id=dataOrig['id'],
                day=len(track.segments),
            ))

        elif oneTrackPer == 1:  # week
            for entry in data:

                updateCounter += 1
                if updateCounter > 1000:
                    print('...{id} | {weeks} weeks...'.format(
                        id=dataOrig['id'],
                        weeks=len(track.segments),
                    ), end='\r')
                    updateCounter = 0

                if len(segment.points) == 0 or timeToDatetime(entry['time']).isocalendar()[1] != segment.points[-1].time.isocalendar()[1]:
                    segment = gpxpy.gpx.GPXTrackSegment()
                    track.segments.append(segment)
                segment.points.append(gpxpy.gpx.GPXTrackPoint(
                    correctLongAndLat(entry['latitude']),
                    correctLongAndLat(entry['longitude']),
                    time=timeToDatetime(entry['time'])
                ))
            print('...{id} | {weeks} weeks...'.format(
                id=dataOrig['id'],
                weeks=len(track.segments),
            ))

        elif oneTrackPer == 2:  # month
            for entry in data:

                updateCounter += 1
                if updateCounter > 1000:
                    print('...' + str(dataOrig['id']) + '||  month ' + str(len(
                        gpx.tracks) + 1) + ' with ' + str(len(segment.points)) + ' points...', end='\r')
                    updateCounter = 0

                if len(segment.points) == 0 or timeToDatetime(entry['time']).month != segment.points[-1].time.month:
                    segment = gpxpy.gpx.GPXTrackSegment()
                    track.segments.append(segment)
                segment.points.append(gpxpy.gpx.GPXTrackPoint(
                    correctLongAndLat(entry['latitude']),
                    correctLongAndLat(entry['longitude']),
                    time=timeToDatetime(entry['time'])
                ))
            print('...' + str(dataOrig['id']) + '||  month ' + str(
                len(gpx.tracks) + 1) + ' with ' + str(len(segment.points)) + ' points...')

        gpxArray.append(gpx)

    print('----- Done -----')
    return gpxArray


def saveGpx(gpx, path, fileName):
    print('Saving gpx...')
    with open(path + fileName, 'w') as f:
        f.write(gpx.to_xml())
    print('----- Done -----')


In [ ]:
# split
splitInto = 0
# -1 = no split
# 0 = year
# 1 = month

# size of track segments
# oneTrackPer = -1
# -1 = no split
# 0 = day
# 1 = week

lightroomMax = 50000

In [ ]:
data = loadOriginal('./source/Location History/', 'Records.json')
# data = loadSimplified('./source/Location History/', 'simplified-Records.json')

In [ ]:
dataSplited = splitData(data, splitInto)

In [ ]:
# print(dataSplited[0]['data'])

# only use first 10 months
dataSplited = dataSplited[:1]

dataGpx = createGfx(dataSplited, oneTrackPer)

for i in range(len(dataGpx)):
    saveGpx(dataGpx[i], './output/', str(dataSplited[i]['id']) + '.gpx')